In [2]:
#!pip install -q wot

In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import ipywidgets as widgets
import wot

Load transport map model and cell sets

In [3]:
tmap_model = wot.tmap.TransportMapModel.from_directory('tmaps_paper/serum_tm_v3_')
cell_sets = wot.io.read_sets('data/cell_sets.gmt', as_dict=True)

Convert cell sets indicating whether a cell at time t is in a cell set

In [4]:
populations = tmap_model.population_from_cell_sets(cell_sets, at_time=18)

Print cell set name and size

In [5]:
for pop in populations:
    print('{}, {}'.format(pop.name, np.count_nonzero(pop.p)))

IPS, 412
Stromal, 1239
Neural, 819
Epithelial, 503
Epithelial_clust, 1013
Trophoblast, 292
OPC, 141
Astrocyte, 112
Neuron, 122
RG_Gdf10, 144
RG_Id3, 160
RG_Neurog2, 21
RG, 255
OtherNeural, 390
TrophoLarger, 1305


Compute trajectories. Trajectory dataset has cells on rows and cell set populations on columns

In [6]:
trajectory_ds = tmap_model.compute_trajectories(populations)

(15, 4068)
(15, 3678)
(15, 4645)
(15, 3422)
(15, 4851)
(15, 3717)
(15, 2723)
(15, 3758)
(15, 2034)
(15, 2145)
(15, 2253)
(15, 2429)
(15, 2164)
(15, 2238)
(15, 1941)
(15, 2051)
(15, 2266)
(15, 2982)
(15, 3088)
(15, 3573)
(15, 3829)
(15, 3815)
(15, 5061)
(15, 6507)
(15, 3111)
(15, 8422)
(15, 6550)
(15, 7227)
(15, 7127)
(15, 8962)
(15, 7355)
(15, 6777)
(15, 6734)
(15, 6981)
(15, 1956)
(15, 3648)
(15, 3449)
(15, 4556)


In [11]:
trajectory_ds[trajectory_ds.obs['day']==4.0].X.sum(axis=0)

ArrayView([0.99999994, 1.0000008 , 0.9999979 , 1.0000002 , 1.        ,
           1.0000001 , 1.        , 1.0000002 , 0.9999998 , 0.9999975 ,
           1.0000011 , 0.99999833, 1.0000014 , 0.9999996 , 1.0000018 ],
          dtype=float32)

Load embedding coordinates

In [17]:
coord_df = pd.read_csv('data/fle_coords.txt', sep='\t', index_col=0)
trajectory_ds.obs = trajectory_ds.obs.join(coord_df)

Visualize trajectories

In [18]:
trajectory_dropdown = widgets.Dropdown(
    options=trajectory_ds.var.index,
    description='Trajectory:'
)

def update_trajectory_vis(name):
    figure = plt.figure(figsize=(10, 10))
    plt.axis('off')
    plt.tight_layout()
    plt.title(name)
    plt.scatter(coord_df['x'], coord_df['y'], c='#f0f0f0',
                   s=4, marker=',', edgecolors='none', alpha=0.8)
    plt.scatter(trajectory_ds.obs['x'], trajectory_ds.obs['y'], c=trajectory_ds[:, name].X,
                   s=4, marker=',', edgecolors='none')
    plt.colorbar().ax.set_title('Trajectory')

widgets.interact(update_trajectory_vis, name=trajectory_dropdown)

interactive(children=(Dropdown(description='Trajectory:', options=('IPS', 'Stromal', 'Neural', 'Epithelial', 'Epithelial_clust', 'Trophoblast', 'OPC', 'Astrocyte', 'Neuron', 'RG_Gdf10', 'RG_Id3', 'RG_Neurog2', 'RG', 'OtherNeural', 'TrophoLarger'), value='IPS'), Output()), _dom_classes=('widget-interact',))

<function __main__.update_trajectory_vis>

Compute cell fates

In [10]:
fate_ds = tmap_model.compute_fates(populations)

Visualize cell fates

In [11]:
fate_dropdown1 = widgets.Dropdown(
    options=fate_ds.var.index,
    description='Fate 1:'
)
fate_dropdown2 = widgets.Dropdown(
    options=fate_ds.var.index,
    description='Fate 2:',
    value=fate_ds.var.index[1]
)

def update_fate_vis(name1, name2):
    figure = plt.figure(figsize=(10, 10))
    fate1 = fate_ds[:, name1].X
    fate2 = fate_ds[:, name2].X
    p = np.log(1e-9 + np.divide(fate1, fate2, out=np.zeros_like(fate1), where=fate2 != 0))
    plt.scatter(fate_ds.obs['day'], p, s=4, marker=',')
    plt.xlabel('Day')
    plt.ylabel('Log Odds')
    plt.title('{} vs. {}'.format(name1, name2))

widgets.interact(update_fate_vis, name1=fate_dropdown1, name2=fate_dropdown2)

interactive(children=(Dropdown(description='Fate 1:', options=('IPS', 'Stromal', 'Neural', 'Epithelial', 'Epithelial_clust', 'Trophoblast', 'OPC', 'Astrocyte', 'Neuron', 'RG_Gdf10', 'RG_Id3', 'RG_Neurog2', 'RG', 'OtherNeural', 'TrophoLarger', 'Other'), value='IPS'), Dropdown(description='Fate 2:', index=1, options=('IPS', 'Stromal', 'Neural', 'Epithelial', 'Epithelial_clust', 'Trophoblast', 'OPC', 'Astrocyte', 'Neuron', 'RG_Gdf10', 'RG_Id3', 'RG_Neurog2', 'RG', 'OtherNeural', 'TrophoLarger', 'Other'), value='Stromal'), Output()), _dom_classes=('widget-interact',))

<function __main__.update_fate_vis>